In [31]:
# Setup the environment
!pip install --upgrade huggingface_hub >> /dev/null
!pip install git+https://github.com/huggingface/transformers -U accelerate peft >> /dev/null
!pip install -i https://pypi.org/simple/ bitsandbytes >> /dev/null

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-ns1b7f8s


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [1]:
import transformers
transformers.__version__

'4.38.1'

In [2]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import torch


In [3]:
# Load the model
model_id = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
tokenizer.model_max_length

1024

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_id,
                                            device_map="auto")

In [ ]:
# tokenizer.add_special_tokens({'pad_token': '[PAD]'})

In [6]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
def test_prompt(prompt, your_model):
    input_ids = tokenizer(prompt, return_tensors='pt')
    logits = your_model.generate(**input_ids.to(device),
                           max_new_tokens=50)
    return tokenizer.decode(logits[0])

In [9]:
text_test = "Give me the top 15 golf equipment company names."

orginal_model_out = test_prompt(text_test, model)

print(orginal_model_out)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Give me the top 15 golf equipment company names.

I'm going to go through the list and give you the top 15 golf equipment companies.

I'm going to go through the list and give you the top 15 golf equipment companies.

I'm going to go through the list


In [19]:
from transformers import (
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from datasets import load_dataset, load_metric, Dataset
import pandas as pd

In [10]:
import json
data = []

with open('/home/aicoder/gitfolder/python_de_learners_data/code_script_notebooks/projects/learning_hf_philosophy/databricks-dolly-15k.jsonl') as file:
    for line in file:
        features = json.loads(line)
        # Filter out examples with context, to keep it simple.
        if features["context"]:
            continue
        # Format the entire example as a single string.
        template = "Instruction:\n{instruction}\n\nResponse:\n{response}"
        data.append(template.format(**features))

In [15]:
data_500 = [x for x in data if len(x) <= 500]

In [15]:
len(data_500)

7339

In [16]:
# take 1500 from that data 
data = data_500[:300]

In [17]:
tokenizer(data[0])

{'input_ids': [6310, 2762, 25, 198, 13828, 318, 257, 4693, 286, 5916, 30, 309, 3008, 393, 371, 3008, 198, 198, 31077, 25, 198, 51, 3008], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [20]:
json_ds = Dataset.from_pandas(pd.DataFrame(data=data))

In [23]:
def tokenize_instruction(row):
    # print(row)
    input_token = tokenizer(row['0'], padding="max_length",
                            truncation=True, max_length=512)
    input_token['labels'] = input_token['input_ids'].copy()
    return input_token

In [24]:
tokenizer.pad_token = tokenizer.eos_token
dataset_tokenized = json_ds.map(tokenize_instruction,
                                remove_columns=["0"],
                                batched=True)

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [25]:
process_ds = dataset_tokenized.train_test_split(test_size=0.2)

In [26]:
process_ds

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 240
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 60
    })
})

In [27]:
process_ds['train'][210]

{'input_ids': [6310,
  2762,
  25,
  198,
  23318,
  502,
  257,
  1351,
  286,
  262,
  513,
  1266,
  4568,
  284,
  466,
  351,
  534,
  3988,
  618,
  262,
  6193,
  318,
  2089,
  2354,
  198,
  198,
  31077,
  25,
  198,
  15580,
  257,
  6285,
  287,
  262,
  2877,
  2119,
  351,
  34794,
  11,
  9582,
  1666,
  290,
  18791,
  13,
  6305,
  40459,
  6918,
  588,
  383,
  1514,
  17300,
  11,
  3837,
  26487,
  290,
  21788,
  11,
  314,
  911,
  81,
  2954,
  262,
  3988,
  13,
  10934,
  257,
  1232,
  78,
  16669,
  13,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  50256,
  502

In [31]:
len(process_ds['train'][210]['labels'])

512

In [32]:
model.get_memory_footprint() / (1024 * 1024 * 1024)

0.47529320418834686

In [34]:
steps_per_epoch=len(process_ds["train"])//(8*1)

steps_per_epoch

30

In [51]:
args = TrainingArguments(
    output_dir="/home/aicoder/training/gpt_minimal",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    evaluation_strategy="steps",
    logging_steps=1,
    eval_steps=steps_per_epoch,		# eval and save once per epoch  	
    save_strategy="epoch",
    num_train_epochs=5,
    lr_scheduler_type="constant",
    # optim="paged_adamw_32bit",
    learning_rate=2e-5,
    group_by_length=True,
    # fp16=True,
    # ddp_find_unused_parameters=False,
    push_to_hub=False,
    report_to="none"
)

In [36]:
datacollator = DataCollatorForLanguageModeling(tokenizer=tokenizer,
                                              mlm=False)

In [52]:
trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    data_collator=datacollator,
    train_dataset=process_ds["train"],
    eval_dataset=process_ds["test"],
    args=args,
)

In [ ]:
# process_ds['train'][0]

In [53]:
trainer.train()

Step,Training Loss,Validation Loss
30,0.297800,4.339381
60,0.204100,4.388601
90,0.128800,4.366632
120,0.668200,4.341177


Checkpoint destination directory /home/aicoder/training/gpt_minimal/checkpoint-120 already exists and is non-empty. Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [39]:
trained_cp = "/home/aicoder/training/gpt_minimal/checkpoint-150/"

In [40]:
trained_model = AutoModelForCausalLM.from_pretrained(trained_cp, device_map="auto")

In [41]:
trained_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [42]:
trained_model_out = test_prompt(text_test, trained_model)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [43]:
print("Base model")
print(orginal_model_out)

print('\n')
print("FT model")
print(trained_model_out)

Base model
Give me the top 15 golf equipment company names.

I'm going to go through the list and give you the top 15 golf equipment companies.

I'm going to go through the list and give you the top 15 golf equipment companies.

I'm going to go through the list


FT model
Give me the top 15 golf equipment company names.

Titleist, Taylormade, Ping, Ping, Cobra, Ping, Cobra, Ping, Ping, Cobra, Ping, Ping, Cobra, Ping, Ping, Cobra, Ping, Ping, Cobra, Ping, Ping, Cobra,


In [46]:
newprompt = "Classify each as either a vodka drink or a gin drink: Screwdriver, Bloody Mary, French 75, White Russian, Gimlet, Cosmopolitan, Tom Collins, Negroni\n\nResponse:\nScrewdrivers, Bloody Mary's, White Russians, and Cosmopolitans are traditionally vodka cocktails, whereas French 75's, Gimlets, Tom Collins', and Negronis are traditionally gin cocktails."
print(test_prompt(newprompt, trained_model))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify each as either a vodka drink or a gin drink: Screwdriver, Bloody Mary, French 75, White Russian, Gimlet, Cosmopolitan, Tom Collins, Negroni

Response:
Screwdrivers, Bloody Mary's, White Russians, and Cosmopolitans are traditionally vodka cocktails, whereas French 75's, Gimlets, Tom Collins', and Negronis are traditionally gin cocktails. Gimlets, Gimlets, Gimlets, and Negronis are traditionally gin cocktails. Gimlets, Gimlets, Gimlets, and Negronis are traditionally vodka cocktails. Gimlets, Gimlets, Gimlets, and Negronis are


In [47]:
print(test_prompt(newprompt, model))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Classify each as either a vodka drink or a gin drink: Screwdriver, Bloody Mary, French 75, White Russian, Gimlet, Cosmopolitan, Tom Collins, Negroni

Response:
Screwdrivers, Bloody Mary's, White Russians, and Cosmopolitans are traditionally vodka cocktails, whereas French 75's, Gimlets, Tom Collins', and Negronis are traditionally gin cocktails. Gimlets, Gimlets, Gimlets, and Negronis are traditionally gin cocktails. Gimlets, Gimlets, Gimlets, and Negronis are traditionally vodka cocktails. Gimlets, Gimlets, Gimlets, and Negronis are
